In [2]:
import matplotlib.pyplot as plt
import matplotlib.image as mping 
import numpy as np
import warnings
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import cv2

#Steps:            Parameters:
#1:Get image       (nothing)         
#2:Get Masked      (Thresholds)             
#3:Darken          (level of change) 
#4:Merge all       (nothing)         
#5:Convert to grey (nothing)  
#6:Blur            (kernel_size)
#7:Canny edge      (LowT,HighT)
#8:Area            (shape, size)
#9:Hough lines     (Threshold, min line length, max line gap, rho, theta)
#10:Fitting        (stuff)
#11:Draw           (Lines to ignore)

In [77]:
def ColorSelect(image, low_rgb, high_rgb):
    mask = np.copy(image)
    thresholds = (image[:,:,0] > high_rgb[0]) \
                   | (image[:,:,0] < low_rgb[0]) \
                   | (image[:,:,1] > high_rgb[1]) \
                   | (image[:,:,1] < low_rgb[1]) \
                   | (image[:,:,2] > high_rgb[2]) \
                   | (image[:,:,2] < low_rgb[2])   
                
    mask[thresholds] = [0,0,0]
    return mask

In [78]:
def RegionOfInterest(original, points):
    
    image = np.copy(original)
    mask = np.zeros_like(original)   
    ignore_mask_color = 255
    
    cv2.fillPoly(mask, points, ignore_mask_color)
    
    masked_image = cv2.bitwise_and(image, mask)
    return masked_image

In [100]:
def LaneLineDetector(original_image):
    #1
    image=np.copy(original_image) 
    height=len(image)
    width=len(image[0])
    
    #2
    yellow_low_thresholds = [180,160,0]
    yellow_high_thresholds = [255,255,140] 
    white_low_thresholds = [190,190,190]
    white_high_thresholds = [255,255,255]
                             
    white_mask = ColorSelect(image,white_low_thresholds,white_high_thresholds)
    yellow_mask = ColorSelect(image,yellow_low_thresholds,yellow_high_thresholds)

    #3 
    change = 100
    darker_image = np.where(image<change,0,image-change)

    #4                             
    whited_image = cv2.bitwise_or(darker_image, white_mask)
    masked_image = cv2.bitwise_or(whited_image, yellow_mask)
   
    #5                        
    grey_image = cv2.cvtColor(masked_image, cv2.COLOR_RGB2GRAY)
    
    #6
    kernel_size = 7                  
    blurred_image = cv2.GaussianBlur(grey_image,(kernel_size, kernel_size), 0)
    
    #7
    low_threshold = 50
    high_threshold = 150    
    edged_image = cv2.Canny(blurred_image, low_threshold, high_threshold)
 
    #8
    p1,p2,p3,p4  = [int(.165*width),int(.925*height)],[int(.469*width),int(.597*height)],[int(.547*width),int(.597*height)],[int(.867*width),int(.924*height)]
    points = np.array([[p1,p2,p3,p4]], dtype=np.int32) 
    region_image= RegionOfInterest(edged_image, points)

    #9
    rho = 1
    theta = np.pi/180
    threshold = 15
    min_line_length = 20
    max_line_gap = 15
    lined_image = np.zeros_like(image)

    lines = cv2.HoughLinesP(region_image, rho, theta, threshold, np.array([]),
                                             min_line_length, max_line_gap)
    #10
    rx,ry,lx,ly = [],[],[],[]
    line_width = 5
    for line in lines:
        for x1,y1,x2,y2 in line: #No lines with little slope. No lines that cross the middle.
            if np.absolute(y1-y2)>(np.absolute(x1-x2)*.4):
                if x1>width/2 and x2>width/2:
                    cv2.line(lined_image,(x1,y1),(x2,y2),(255,0,0),line_width)
                    rx.append(x1)
                    rx.append(x2)
                    ry.append(y1)
                    ry.append(y2)
                elif x1<width/2 and x2<width/2:
                    cv2.line(lined_image,(x1,y1),(x2,y2),(255,0,0),line_width)
                    lx.append(x1)
                    lx.append(x2)
                    ly.append(y1)
                    ly.append(y2)

    try:
        
        r = np.polyfit(rx, ry, 1)
        l = np.polyfit(lx, ly, 1)
        print([r,l])
        #11
        fr = np.poly1d(r)
        fl = np.poly1d(l)

        pxl,pxr = 21,57
        shift = 20
        w = width*.01
        for x in range(23,53,2): #left
            cv2.line(lined_image,(int(w*pxl),int(fl(w*pxl)-shift*.5)),(int(w*x),int(fl(w*x)-shift*.5)),(255,255,0),line_width)
            pxl=x
        for x in range(59,89,2): #right
            cv2.line(lined_image,(int(w*pxr),int(height-fl(w*pxr)+5*shift)),(int(w*x),int(height-fl(w*x)+5*shift)),(255,255,0),line_width)
            pxr=x
    except:
        pass
    

    laned_image = cv2.addWeighted(image, 0.8, lined_image, 1, 0) 

    return laned_image 

In [101]:
final_clip_output = 'final_output1.mp4'
test_clip = VideoFileClip("test.mp4")
final_clip = test_clip.fl_image(LaneLineDetector)
%time final_clip.write_videofile(final_clip_output, audio=False)

[array([  0.61050051,  17.23768547]), array([ -6.71950289e-01,   8.62319908e+02])]
[MoviePy] >>>> Building video final_output1.mp4
[MoviePy] Writing video final_output1.mp4




  0%|          | 0/251 [00:00<?, ?it/s]

  0%|          | 1/251 [00:00<00:45,  5.49it/s]

[array([  0.61050051,  17.23768547]), array([ -6.71950289e-01,   8.62319908e+02])]
[array([ 0.61891808,  8.56389371]), array([ -5.97231760e-01,   8.20413760e+02])]




  1%|          | 2/251 [00:00<00:47,  5.28it/s]

  1%|          | 3/251 [00:00<00:47,  5.26it/s]

[array([  0.61236525,  12.71750319]), array([ -6.72648717e-01,   8.62961486e+02])]


...



 99%|█████████▉| 249/251 [00:44<00:00,  5.03it/s]

100%|█████████▉| 250/251 [00:44<00:00,  5.24it/s]

[array([  0.60331532,  20.34062796]), array([ -7.12963455e-01,   8.90108490e+02])]
[array([  0.65783753, -20.12540046]), array([ -7.29992210e-01,   9.01038512e+02])]




100%|██████████| 251/251 [00:44<00:00,  4.19it/s]

[array([ 0.64130973, -8.46998991]), array([ -7.26857605e-01,   8.97223489e+02])]


[MoviePy] Done.
[MoviePy] >>>> Video ready: final_output1.mp4 

CPU times: user 2min 3s, sys: 9.41 s, total: 2min 13s
Wall time: 47.7 s


In [3]:
HTML("""
<video width="960" height="520" controls>
  <source src="{0}" type="video/mp4">
</video>
""".format(final_clip_output))

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



NameError: name 'final_clip_output' is not defined

In [103]:
white_output = 'white_output1.mp4'
testw_clip = VideoFileClip("solidWhiteRight.mp4")
white_clip = testw_clip.fl_image(LaneLineDetector)
%time white_clip.write_videofile(white_output, audio=False)

[array([ 0.61727469,  8.80401271]), array([  -0.71693555,  651.5242781 ])]
[MoviePy] >>>> Building video white_output1.mp4
[MoviePy] Writing video white_output1.mp4




  0%|          | 0/222 [00:00<?, ?it/s]

[array([ 0.61727469,  8.80401271]), array([  -0.71693555,  651.5242781 ])]




  0%|          | 1/222 [00:00<00:35,  6.20it/s]

  1%|          | 2/222 [00:00<00:34,  6.32it/s]

[array([ 0.62904752,  1.60268295]), array([  -0.71892147,  651.67212965])]


...

[array([  0.56618031,  35.22763905]), array([  -0.79286662,  686.5335123 ])]
[array([  0.56202567,  38.36785764]), array([  -0.82001736,  693.6549031 ])]




 99%|█████████▊| 219/222 [00:23<00:00,  9.22it/s]

[array([  0.57693608,  28.99413473]), array([  -0.73556536,  671.35888327])]
[array([  0.57415549,  30.87390889]), array([  -0.76092389,  675.16020508])]
[array([  0.57500491,  30.34524668]), array([  -0.69906929,  646.64150293])]




100%|█████████▉| 221/222 [00:23<00:00, 10.05it/s]



[MoviePy] Done.
[MoviePy] >>>> Video ready: white_output1.mp4 

CPU times: user 1min 8s, sys: 4.5 s, total: 1min 12s
Wall time: 24.6 s


In [104]:
HTML("""
<video width="960" height="520" controls>
  <source src="{0}" type="video/mp4">
</video>
""".format(white_output))

In [105]:
yellow_output = 'yellow_output1.mp4'
testy_clip = VideoFileClip("solidYellowLeft.mp4")
yellow_clip = testy_clip.fl_image(LaneLineDetector)
%time yellow_clip.write_videofile(yellow_output, audio=False)

[array([  0.68934276, -32.01146541]), array([  -0.62580396,  611.55926607])]
[MoviePy] >>>> Building video yellow_output1.mp4
[MoviePy] Writing video yellow_output1.mp4




  0%|          | 0/682 [00:00<?, ?it/s]

  0%|          | 2/682 [00:00<00:50, 13.46it/s]

[array([  0.68934276, -32.01146541]), array([  -0.62580396,  611.55926607])]
[array([  0.68735834, -29.76692379]), array([  -0.63033525,  614.3376622 ])]


[array([  0.67897649, -25.11654336]), array([  -0.6168337,  609.343378 ])]




  1%|          | 4/682 [00:00<00:48, 13.92it/s]

...

[array([  0.58093683,  31.53363195]), array([  -0.72081317,  657.16586255])]
[array([  0.60060662,  21.45149759]), array([  -0.71717654,  655.50751001])]
[array([  0.58454918,  31.27907064]), array([  -0.71794872,  655.35897436])]




100%|█████████▉| 680/682 [01:08<00:00, 10.51it/s]

[array([  0.60482468,  18.96339121]), array([  -0.72537992,  658.86215257])]
[array([  0.58789592,  29.39087472]), array([  -0.72946353,  661.55561984])]
[array([  0.58418602,  32.51590612]), array([  -0.72640319,  658.82718484])]




100%|█████████▉| 681/682 [01:09<00:00,  9.84it/s]

[MoviePy] Done.
[MoviePy] >>>> Video ready: yellow_output1.mp4 

CPU times: user 3min 25s, sys: 14.1 s, total: 3min 39s
Wall time: 1min 10s


In [106]:
HTML("""
<video width="960" height="520" controls>
  <source src="{0}" type="video/mp4">
</video>
""".format(yellow_output))